In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/')

Mounted at /content/drive


In [ ]:
import torch.nn as nn
import pandas as pd
import torch
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

True

In [ ]:
train_df = pd.read_csv("mnist/sign_mnist_train.csv")
valid_df = pd.read_csv("mnist/sign_mnist_valid.csv")

In [ ]:
sample_df = train_df.head().copy()  # Grab the top 5 rows
sample_df.pop('label')
sample_x = sample_df.values
sample_x

array([[107, 118, 127, ..., 204, 203, 202],
       [155, 157, 156, ..., 103, 135, 149],
       [187, 188, 188, ..., 195, 194, 195],
       [211, 211, 212, ..., 222, 229, 163],
       [164, 167, 170, ..., 163, 164, 179]])

In [ ]:
sample_x.shape

(5, 784)

In [ ]:
IMG_HEIGHT = 28
IMG_WIDTH = 28
IMG_CHS = 1

sample_x = sample_x.reshape(-1, IMG_CHS, IMG_HEIGHT, IMG_WIDTH)
sample_x.shape

(5, 1, 28, 28)

In [ ]:
class MyDataset(Dataset):
  def __init__(self, base_df):
    x_df = base_df.copy()
    y_df = x_df.pop('label')
    x_df = x_df.values / 255
    x_df = x_df.reshape(-1, IMG_CHS, IMG_WIDTH, IMG_HEIGHT)

    self.xs = torch.tensor(x_df).float().to(device)
    self.ys = torch.tensor(y_df).to(device)

  def __getitem__(self,idx):
    x = self.xs[idx]
    y = self.ys[idx]
    return x,y

  def __len__(self):
    return len(self.xs)

In [ ]:
BATCH_SIZE = 32

train_data = MyDataset(train_df)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
train_N = len(train_loader.dataset)

valid_data = MyDataset(valid_df)
valid_loader = DataLoader(valid_data, batch_size=BATCH_SIZE)
valid_N = len(valid_loader.dataset)

In [ ]:
batch = next(iter(train_loader))
batch

[tensor([[[[0.8549, 0.8549, 0.8510,  ..., 0.8549, 0.8588, 0.8549],
           [0.8549, 0.8549, 0.8510,  ..., 0.8588, 0.8549, 0.8510],
           [0.8588, 0.8588, 0.8588,  ..., 0.8627, 0.8588, 0.8549],
           ...,
           [0.5529, 0.5529, 0.5529,  ..., 0.7843, 0.8824, 0.8745],
           [0.5529, 0.5490, 0.5490,  ..., 0.7647, 0.7569, 0.8431],
           [0.5569, 0.5490, 0.5451,  ..., 0.8941, 0.7882, 0.6824]]],
 
 
         [[[0.5647, 0.5843, 0.5922,  ..., 0.6353, 0.6275, 0.6275],
           [0.5765, 0.5922, 0.6039,  ..., 0.6471, 0.6392, 0.6392],
           [0.5882, 0.6078, 0.6157,  ..., 0.6588, 0.6549, 0.6471],
           ...,
           [0.7059, 0.7216, 0.7373,  ..., 0.7882, 0.7843, 0.7804],
           [0.7059, 0.7255, 0.7451,  ..., 0.7882, 0.7843, 0.7765],
           [0.7098, 0.7333, 0.7529,  ..., 0.7922, 0.7882, 0.7804]]],
 
 
         [[[0.0980, 0.1333, 0.1529,  ..., 0.5569, 0.5647, 0.5686],
           [0.1020, 0.1333, 0.1529,  ..., 0.5647, 0.5647, 0.5725],
           [0.1137

In [ ]:
batch[0].shape

torch.Size([32, 1, 28, 28])

In [ ]:
batch[1].shape

torch.Size([32])

### Create CNN Model

In [ ]:
n_classes = 24
kernel_size = 3
flattened_img_size = 75 * 3 * 3

model = nn.Sequential(
    # First convolution
    nn.Conv2d(IMG_CHS, 25, kernel_size, stride=1, padding=1), # 25 x 28 x 28
    nn.BatchNorm2d(25),
    nn.ReLU(),
    nn.MaxPool2d(2, stride=2), # 25 x 14 x 14

    # second convolution
    nn.Conv2d(25, 50, kernel_size, stride=1, padding=1), # 50 x 14 x 14
    nn.BatchNorm2d(50),
    nn.ReLU(),
    nn.Dropout(.2),
    nn.MaxPool2d(2, stride=2), # 50 x 7 x 7

    # third convolution
    nn.Conv2d(50, 75, kernel_size, stride=1, padding=1), # 75 x 7 x 7
    nn.BatchNorm2d(75),
    nn.ReLU(),
    nn.MaxPool2d(2, stride=2),  # 75 x 3 x 3

    # flatten to dense
    nn.Flatten(),
    nn.Linear(flattened_img_size, 512),
    nn.Dropout(.3),
    nn.ReLU(),
    nn.Linear(512, n_classes)




)

In [ ]:
model = torch.compile(model.to(device))
model

OptimizedModule(
  (_orig_mod): Sequential(
    (0): Conv2d(1, 25, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(25, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(50, 75, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): BatchNorm2d(75, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): Flatten(start_dim=1, end_dim=-1)
    (14): Linear(in_features=675, out_features=512, bias=True)
    (15): Dropout

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters())

In [ ]:
def get_batch_accuracy(output, y, N):
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct / N

In [ ]:
# SOLUTION
def validate():
    loss = 0
    accuracy = 0

    model.eval()
    with torch.no_grad():
        for x, y in valid_loader:
            output = model(x)

            loss += loss_function(output, y).item()
            accuracy += get_batch_accuracy(output, y, valid_N)
    print('Valid - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [ ]:
# SOLUTION
def train():
    loss = 0
    accuracy = 0

    model.train()
    for x, y in train_loader:
        output = model(x)
        optimizer.zero_grad()
        batch_loss = loss_function(output, y)
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y, train_N)
    print('Train - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [ ]:
epochs = 20

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    train()
    validate()

Epoch: 0
Train - Loss: 2.2750 Accuracy: 0.9993
Valid - Loss: 18.6590 Accuracy: 0.9813
Epoch: 1
Train - Loss: 5.7010 Accuracy: 0.9981
Valid - Loss: 55.2708 Accuracy: 0.9361
Epoch: 2
Train - Loss: 4.3507 Accuracy: 0.9987
Valid - Loss: 22.2589 Accuracy: 0.9734
Epoch: 3
Train - Loss: 1.1986 Accuracy: 0.9996
Valid - Loss: 23.5790 Accuracy: 0.9770
Epoch: 4
Train - Loss: 2.8386 Accuracy: 0.9991
Valid - Loss: 66.3853 Accuracy: 0.9550
Epoch: 5
Train - Loss: 4.1812 Accuracy: 0.9988
Valid - Loss: 43.1090 Accuracy: 0.9621
Epoch: 6
Train - Loss: 0.4591 Accuracy: 0.9999
Valid - Loss: 41.3800 Accuracy: 0.9690
Epoch: 7
Train - Loss: 3.5612 Accuracy: 0.9993
Valid - Loss: 50.8167 Accuracy: 0.9473
Epoch: 8
Train - Loss: 2.3675 Accuracy: 0.9991
Valid - Loss: 31.0296 Accuracy: 0.9678
Epoch: 9
Train - Loss: 0.6095 Accuracy: 0.9998
Valid - Loss: 42.8845 Accuracy: 0.9596
Epoch: 10
Train - Loss: 5.1661 Accuracy: 0.9984
Valid - Loss: 36.5354 Accuracy: 0.9791
Epoch: 11
Train - Loss: 1.2967 Accuracy: 0.9996
Valid